In [1]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [2]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [3]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [4]:
W = 250
hours = 24

insample_scenarios = Dict()
out_of_sample_scenarios = Dict()
seeds = [1,2,3,4,5,6,7,8,9,10]

for seed in seeds
    Random.seed!(seed)
    selected_scenarios = rand(1:n_scenarios, W)
    scenarios = Dict()
    counter = 1
    for i in selected_scenarios
        scenarios[counter] = all_scenarios[i]
        counter += 1
    end
    insample_scenarios[seed] = scenarios

    # create the list of index of scenarios out of sample
    out_of_sample = []
    for i in 1:n_scenarios
        if i ∉ selected_scenarios
            push!(out_of_sample, i)
        end
    end

    unselected_scenarios = Dict()
    counter = 1
    for i in out_of_sample
        unselected_scenarios[counter] = all_scenarios[i]
        counter += 1
    end
    out_of_sample_scenarios[seed] = unselected_scenarios
end

alpha = 0.9
beta = 0.8
objectiv_values = []
exp_profits = []
profits_in = []
CVaR_values = []


Any[]

In [5]:
expected_profit_list = []
#create df with seeds as column and add the p_DA_values for each seed and definec the columns
bids_df = DataFrame()

for seed in seeds[1:3]
    scenarios = insample_scenarios[seed]

    # Create a new model with GLPK solver
    model = Model(GLPK.Optimizer)
    unregister(model, :p_DA)

    # Define the decision variables for hour
    @variable(model, p_DA[1:hours])
    @variable(model, delta[1:W,1:hours])
    @variable(model, delta_up[1:W,1:hours])
    @variable(model, delta_down[1:W,1:hours])
    @variable(model, zeta)
    @variable(model, eta[1:W] >= 0)

    # Define the objective function
    @objective(model, Max, (1-beta) *  sum(1/W*(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
        + beta  * (zeta - (1/(1-alpha)) * sum(1/W * eta[i] for i in 1:W))) for i in 1:W, hour in 1:hours))

    # Define the constraints
    @constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
    @constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
    @constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
    #@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)

    @constraint(model, [i in 1:W], -1 * sum(scenarios[i][hour,"price"] * p_DA[hour]
    + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
    - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for hour in 1:hours) + zeta - eta[i] <= 0)



    # Solve the optimization problem
    optimize!(model)

    # save the p_DA values
    p_DA_values = value.(p_DA)

    exp_profit = value.(sum(1/W.*(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for i in 1:W, hour in 1:hours)))

    # save the profit of each scenario
    for i in 1:W
        push!(profits_in, value.(sum(scenarios[i][hour,"price"] * p_DA[hour]
        + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
        - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2) for hour in 1:hours)))
    end


    #println(model)

    # Print the termination status
    status = termination_status(model)
    if status == MOI.OPTIMAL
        println("Expected profit ", exp_profit)
        println("p_DA: ", value.(p_DA))
        #add the p_DA values to the df
        bids_df[!, string(seed)] = value.(p_DA)
        
    else
        println("No optimal solution found")
    end
end

Expected profit 125415.10715913188
p_DA: [22.33724183870968, 14.389693451612905, 15.949935451612903, 13.469580564516129, 9.242903161290323, 5.90827414516129, 145.9269354516129, 2.7339112741935483, 1.1038967741935486, 146.94499998387096, 0.7919354838709678, 0.7419306451612904, 142.91516125806453, 1.167741935483871, 135.9887096612903, 2.3072451612903224, 146.0248386935484, 149.60145158064518, 5.493548387096774, 152.74596772580645, 4.493516129032258, 4.803145161290321, 6.808774177419355, 149.57870966129033]
Expected profit 132429.67084251664
p_DA: [22.33724183870968, 14.389693451612905, 15.949935451612903, 13.469580564516129, 9.242903161290323, 5.90827414516129, 145.9269354516129, 2.7339112741935483, 1.1038967741935415, 146.94499998387096, 0.7919354838709678, 0.7419306451612904, 142.91516125806453, 1.167741935483871, 135.9887096612903, 2.3072451612903224, 146.0248386935484, 149.60145158064518, 5.493548387096774, 152.74596772580645, 4.493516129032258, 4.803145161290321, 6.808774177419355, 

In [6]:
bids_df

Row,1,2,3
,Float64,Float64,Float64
1,22.3372,22.3372,22.3372
2,14.3897,14.3897,14.3897
3,15.9499,15.9499,15.9499
4,13.4696,13.4696,13.4696
5,9.2429,9.2429,9.2429
6,5.90827,5.90827,5.90827
7,145.927,145.927,145.927
8,2.73391,2.73391,2.73391
9,1.1039,1.1039,1.1039


In [7]:
W_new = n_scenarios - W
hours = 24
profits_out = []

# calculate the expected profit for the out of sample scenarios
exp_profit = 0
for i in 1:W_new
    profit_day = 0
    for hour in 1:hours
        profit = 0
        profit += unselected_scenarios[1][hour,"price"] * p_DA_values[hour] # should the day ahead price be fixed to the first scenario? So instead of i, 1?
        if unselected_scenarios[i][hour,"wind power"] > p_DA_values[hour]
            profit += unselected_scenarios[i][hour,"price"] * (unselected_scenarios[i][hour,"wind power"] - p_DA_values[hour]) * (unselected_scenarios[i][hour,"grid_excess"]*0.9 + (1-unselected_scenarios[i][hour,"grid_excess"])*1)
        else
            profit -= unselected_scenarios[i][hour,"price"] * (p_DA_values[hour] - unselected_scenarios[i][hour,"wind power"]) * (unselected_scenarios[i][hour,"grid_excess"]*1 + (1-unselected_scenarios[i][hour,"grid_excess"])*1.2)
        end
        profit_day += profit
    end
    push!(profits_out, profit_day)
    exp_profit += profit_day
end

exp_profit = exp_profit / W_new



UndefVarError: UndefVarError: `unselected_scenarios` not defined

In [8]:
# create a dataframe with the profits and if the scenanrio is in or out of sample
df = DataFrame(Scenario = 1:W + W_new, Profit = [profits_in; profits_out], In_Out = [fill("In",W); fill("Out",W_new)])  

# save the data
CSV.write("1_4_results/profits.csv", df)




DimensionMismatch: DimensionMismatch: column :Scenario has length 1200 and column :Profit has length 750